In [1]:
import pandas as pd
import numpy as np
hourly_load = pd.read_csv('../clean_data/hourly_load_act.csv', parse_dates=['interval_start_local'])
solar = pd.read_csv('../clean_data/ercot_solar_actuals_allzones_2023_2024.csv', parse_dates=['interval_start_local'])
wind = pd.read_csv('../clean_data/ercot_wind_actuals_hourly_2023_2024.csv', parse_dates=['interval_start_local'])
lmp = pd.read_csv('../clean_data/LMP_2023_2024_Hubs.csv', parse_dates=['datetime'])
weather = pd.read_csv('../clean_data/weather_hourly.csv', parse_dates=['datetime'])


In [14]:
hourly_load.columns
hourly_load.dtypes

interval_start_local    datetime64[ns]
load                           float64
dtype: object

## Normalize datetime columns

Define a small helper to robustly parse and normalize datetime-like columns across datasets.
This will convert timezone-offset strings to `America/Chicago` local time and produce naive datetimes for consistent merging.

In [18]:
import pandas as pd
def ensure_localized(df, col, tz='America/Chicago', make_naive=True):
    """Parse `col` in `df` robustly and convert to timezone `tz`.
    - Tries parsing with `utc=True` first (handles strings with offsets like `-06:00`).
    - Falls back to parsing without `utc` and then localizing naive timestamps.
    - Converts to `tz` and (optionally) drops tz info producing naive local times.
    """
    if col not in df.columns:
        print(f"column not found: {col}")
        return
    # First try: parse with utc=True (works for strings that include offsets)
    s = pd.to_datetime(df[col], errors='coerce', utc=True)
    # If parsing produced all NaT or resulted in tz-naive series, try fallback parse
    if s.isna().all() or (getattr(s.dt, 'tz', None) is None):
        s = pd.to_datetime(df[col], errors='coerce')
        # If still naive (no tz), localize to tz
        if getattr(s.dt, 'tz', None) is None:
            try:
                s = s.dt.tz_localize(tz)
            except Exception:
                # some strings may already include offsets; reparse with utc
                s = pd.to_datetime(df[col], errors='coerce', utc=True)
    # At this point s should be tz-aware (UTC or localized)
    if getattr(s.dt, 'tz', None) is not None:
        try:
            s = s.dt.tz_convert(tz)
        except Exception:
            # if conversion fails, keep as-is
            pass
    # Optionally drop tz info to produce naive local timestamps
    if make_naive and getattr(s.dt, 'tz', None) is not None:
        s = s.dt.tz_localize(None)
    df[col] = s

# Apply to the dataframes already loaded above
tz = 'America/Chicago'
ensure_localized(hourly_load, 'interval_start_local', tz=tz, make_naive=True)
ensure_localized(solar, 'interval_start_local', tz=tz, make_naive=True)
ensure_localized(wind, 'interval_start_local', tz=tz, make_naive=True)
ensure_localized(lmp, 'datetime', tz=tz, make_naive=True)
ensure_localized(weather, 'datetime', tz=tz, make_naive=True)

# Quick verification prints
for name, df, col in [
    ('hourly_load', hourly_load, 'interval_start_local'),
    ('solar', solar, 'interval_start_local'),
    ('wind', wind, 'interval_start_local'),
    ('lmp', lmp, 'datetime'),
    ('weather', weather, 'datetime'),
]:
    if col in df.columns:
        print(f"{name} -> {col} dtype:", df[col].dtype, "nulls:", df[col].isna().sum())
        display(df[[col]].head(3))
    else:
        print(f"{name} missing column {col}")

column not found: datetime
column not found: datetime
hourly_load -> interval_start_local dtype: datetime64[ns] nulls: 0


,interval_start_local
0,2020-12-31 18:00:00
1,2020-12-31 19:00:00
2,2020-12-31 20:00:00


solar -> interval_start_local dtype: datetime64[ns] nulls: 0


,interval_start_local
0,2023-01-01 00:00:00
1,2023-01-01 01:00:00
2,2023-01-01 02:00:00


wind -> interval_start_local dtype: datetime64[ns] nulls: 0


,interval_start_local
0,2023-01-01 00:00:00
1,2023-01-01 01:00:00
2,2023-01-01 02:00:00


lmp missing column datetime
weather missing column datetime


In [19]:
solar.columns
solar.dtypes
# Convert column to datetime


interval_start_local    datetime64[ns]
gen_system_wide                float64
gen_centerwest                 float64
gen_northwest                  float64
gen_fareast                    float64
gen_southeast                  float64
gen_centereast                 float64
dtype: object

In [20]:
wind.columns
wind.drop(columns=['interval_start_utc', 'interval_end_utc', 'interval_end_local'], inplace=True)

KeyError: "['interval_start_utc', 'interval_end_utc', 'interval_end_local'] not found in axis"

In [21]:
lmp.columns
# Column types
lmp.dtypes

interval_start_local    datetime64[ns]
HB_BUSAVG                      float64
HB_HOUSTON                     float64
HB_HUBAVG                      float64
HB_NORTH                       float64
HB_PAN                         float64
HB_SOUTH                       float64
HB_WEST                        float64
dtype: object

In [22]:
weather.columns

Index(['interval_start_local', 'TEMP_C', 'TEMP_qc', 'DEW_C', 'DEW_qc',
       'SLP_hPa', 'SLP_qc', 'WIND_DIR_deg', 'WIND_DIR_qc', 'WIND_SPD_ms',
       'WIND_SPD_qc'],
      dtype='object')

In [23]:
weather.rename(columns={'datetime':'interval_start_local'}, inplace=True)
lmp.rename(columns={'datetime':'interval_start_local'}, inplace=True)


In [24]:
merged = hourly_load.merge(solar, on='interval_start_local', how='left')
merged = merged.merge(wind, on='interval_start_local', how='left')
merged = merged.merge(lmp, on='interval_start_local', how='left')
merged = merged.merge(weather, on='interval_start_local', how='left')

In [25]:
merged.columns

Index(['interval_start_local', 'load', 'gen_system_wide_x', 'gen_centerwest',
       'gen_northwest', 'gen_fareast', 'gen_southeast', 'gen_centereast',
       'publish_time_utc', 'publish_time_local', 'gen_system_wide_y',
       'gen_lz_south_houston', 'gen_lz_west', 'gen_lz_north', 'HB_BUSAVG',
       'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_PAN', 'HB_SOUTH', 'HB_WEST',
       'TEMP_C', 'TEMP_qc', 'DEW_C', 'DEW_qc', 'SLP_hPa', 'SLP_qc',
       'WIND_DIR_deg', 'WIND_DIR_qc', 'WIND_SPD_ms', 'WIND_SPD_qc'],
      dtype='object')

In [27]:
merged.shape

(35040, 31)

In [28]:
merged.isna().sum()

interval_start_local        0
load                        7
gen_system_wide_x       17527
gen_centerwest          17527
gen_northwest           17527
gen_fareast             17527
gen_southeast           17527
gen_centereast          17527
publish_time_utc        17526
publish_time_local      17526
gen_system_wide_y       17529
gen_lz_south_houston    17529
gen_lz_west             17529
gen_lz_north            17529
HB_BUSAVG               17526
HB_HOUSTON              17526
HB_HUBAVG               17526
HB_NORTH                17526
HB_PAN                  17526
HB_SOUTH                17526
HB_WEST                 17526
TEMP_C                     13
TEMP_qc                    20
DEW_C                      16
DEW_qc                     20
SLP_hPa                    49
SLP_qc                     11
WIND_DIR_deg             1898
WIND_DIR_qc                11
WIND_SPD_ms                16
WIND_SPD_qc                14
dtype: int64

In [29]:
# Drop rows with HB_BUSAVG is NA
merged = merged[~merged['HB_BUSAVG'].isna()]

In [30]:
# Check continuity
merged.shape

(17514, 31)

In [31]:
merged.columns

Index(['interval_start_local', 'load', 'gen_system_wide_x', 'gen_centerwest',
       'gen_northwest', 'gen_fareast', 'gen_southeast', 'gen_centereast',
       'publish_time_utc', 'publish_time_local', 'gen_system_wide_y',
       'gen_lz_south_houston', 'gen_lz_west', 'gen_lz_north', 'HB_BUSAVG',
       'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_PAN', 'HB_SOUTH', 'HB_WEST',
       'TEMP_C', 'TEMP_qc', 'DEW_C', 'DEW_qc', 'SLP_hPa', 'SLP_qc',
       'WIND_DIR_deg', 'WIND_DIR_qc', 'WIND_SPD_ms', 'WIND_SPD_qc'],
      dtype='object')

In [32]:
merged.head()


,interval_start_local,load,gen_system_wide_x,gen_centerwest,gen_northwest,gen_fareast,gen_southeast,gen_centereast,publish_time_utc,publish_time_local,...,TEMP_C,TEMP_qc,DEW_C,DEW_qc,SLP_hPa,SLP_qc,WIND_DIR_deg,WIND_DIR_qc,WIND_SPD_ms,WIND_SPD_qc
17526,2023-01-01 00:00:00,34969.250000,0.45,0.01,0.0,0.36,0.0,0.07,2023-01-03 06:55:33+00:00,2023-01-03T00:55:33-06:00,...,17.75,3.0,6.7,3.0,1007.75,3.0,185.0,3.0,4.1,3.0
17527,2023-01-01 01:00:00,35573.500000,0.46,0.01,0.0,0.37,0.0,0.07,2023-01-03 07:55:35+00:00,2023-01-03T01:55:35-06:00,...,16.70,5.0,7.2,5.0,1007.90,5.0,180.0,5.0,4.6,5.0
17528,2023-01-01 02:00:00,36279.750000,0.45,0.01,0.0,0.36,0.0,0.07,2023-01-03 08:55:33+00:00,2023-01-03T02:55:33-06:00,...,16.10,5.0,7.2,5.0,1008.70,5.0,170.0,5.0,3.1,5.0
17529,2023-01-01 03:00:00,36765.833333,0.46,0.01,0.0,0.37,0.0,0.07,2023-01-03 09:55:29+00:00,2023-01-03T03:55:29-06:00,...,15.85,3.0,7.2,3.0,1008.80,3.0,175.0,3.0,3.6,3.0
17530,2023-01-01 04:00:00,37049.916667,0.45,0.01,0.0,0.36,0.0,0.07,2023-01-03 10:55:19+00:00,2023-01-03T04:55:19-06:00,...,15.00,5.0,7.2,5.0,1009.40,5.0,190.0,5.0,4.6,5.0


In [37]:
# Diagnostic: inspect the in-memory `merged` DataFrame (cell 17 context)
import pandas as pd
# ensure datetime column is parsed if present
dt_col = None
for c in ['interval_start_local','datetime','start','timestamp']:
    if c in merged.columns:
        dt_col = c
        break
if dt_col is not None:
    merged[dt_col] = pd.to_datetime(merged[dt_col], errors='coerce')
print('Columns in merged (count={}):'.format(len(merged.columns)))
print(list(merged.columns))
# Find candidate HB/LMP columns by keyword
candidates = [c for c in merged.columns if any(k in c.lower() for k in ('hb','busavg','lmp','hub','bus'))]
print('LMP/HB candidate columns:', candidates)
# If HB_BUSAVG exists, show its stats; otherwise show stats for candidates
target = 'HB_BUSAVG' if 'HB_BUSAVG' in merged.columns else (candidates[0] if candidates else None)
if target is None:
    print('No HB/LMP-like column found in `merged`.')
else:
    print('Using column for LMP analysis:', target)
    ser = merged[target]
    print(ser.describe())
    print('nulls:', ser.isna().sum(), 'of', len(ser))
    # show sample values where not null
    print('Sample non-null values:')
    display(merged.loc[merged[target].notna(), [dt_col, target]].head(10))
    # continuity check on rows where this LMP is present
    if dt_col is not None:
        df_present = merged.loc[merged[target].notna()].copy()
        df_present = df_present.sort_values(dt_col)
        s = pd.to_datetime(df_present[dt_col])
        diffs = s.diff().dt.total_seconds()/3600.0
        gaps = (diffs>1.001).sum()
        print('Rows with LMP present:', len(df_present))
        print('Gaps (diff>1.001h) among those rows:', gaps)
        # per-day completeness
        sdf = pd.DataFrame({dt_col: s})
        sdf['date'] = sdf[dt_col].dt.date
        sdf['hour'] = sdf[dt_col].dt.hour
        days = []
        for date, g in sdf.groupby('date'):
            hours = sorted(g['hour'].unique())
            days.append((date, len(hours)))
        full_days = sum(1 for d,n in days if n==24)
        print('Days with full 24 hours for rows where LMP present:', full_days, 'of', len(days))
    else:
        print('Skipping LMP continuity check because no datetime column found in merged.')

Columns in merged (count=31):
['interval_start_local', 'load', 'gen_system_wide_x', 'gen_centerwest', 'gen_northwest', 'gen_fareast', 'gen_southeast', 'gen_centereast', 'publish_time_utc', 'publish_time_local', 'gen_system_wide_y', 'gen_lz_south_houston', 'gen_lz_west', 'gen_lz_north', 'HB_BUSAVG', 'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_PAN', 'HB_SOUTH', 'HB_WEST', 'TEMP_C', 'TEMP_qc', 'DEW_C', 'DEW_qc', 'SLP_hPa', 'SLP_qc', 'WIND_DIR_deg', 'WIND_DIR_qc', 'WIND_SPD_ms', 'WIND_SPD_qc']
LMP/HB candidate columns: ['HB_BUSAVG', 'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_PAN', 'HB_SOUTH', 'HB_WEST']
Using column for LMP analysis: HB_BUSAVG
count    17514.000000
mean        35.690520
std        177.204636
min        -31.770000
25%         14.700000
50%         20.340000
75%         26.610000
max       5054.490000
Name: HB_BUSAVG, dtype: float64
nulls: 0 of 17514
Sample non-null values:


,interval_start_local,HB_BUSAVG
17526,2023-01-01 00:00:00,-2.56
17527,2023-01-01 01:00:00,-1.49
17528,2023-01-01 02:00:00,-1.00
17529,2023-01-01 03:00:00,-0.05
17530,2023-01-01 04:00:00,0.00
17531,2023-01-01 05:00:00,-1.55
17532,2023-01-01 06:00:00,2.45
17533,2023-01-01 07:00:00,11.93
17534,2023-01-01 08:00:00,22.51
17535,2023-01-01 09:00:00,10.88


Rows with LMP present: 17514
Gaps (diff>1.001h) among those rows: 2
Days with full 24 hours for rows where LMP present: 727 of 730


In [38]:
merged.to_csv('../clean_data/train_data.csv', index=False)